In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
file_path = 'D:/EDM/数据集相关/22级学生行为数据集/DataSet_V5_new_filled.xlsx'
df = pd.read_excel(file_path)
df = df.drop(columns=['xh'])

In [ ]:
from sklearn.feature_selection import VarianceThreshold

# 进行方差选择法特征选择
selector = VarianceThreshold(threshold=0.01)
df_new= selector.fit_transform(df)

# 输出选择的特征
print("原始特征数：", df.shape[1])
print("过滤后的特征数：", df_new.shape[1])
selected_features = df.columns[selector.get_support()]
print("选择的特征：", selected_features)
# 将 df_new 转换为 DataFrame，并使用原始的列名
df_new = pd.DataFrame(df_new, columns=selected_features)

In [3]:
X = df_new.drop(columns=['label'])
# X = df.drop(columns=['xh','label'])
y = df_new['label']

scaler = StandardScaler()
min_max_scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X = min_max_scaler.fit_transform(X)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=0.7, stratify=y, random_state=42)

# 第二次划分，得到测试集 (20%) 和验证集 (10%)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.3, stratify=y_temp, random_state=42)

# X_train, X_test, y_train, y_test = train_test_split(df[df.columns[1:31]], df["label"], train_size=0.8, stratify=df["label"])

使用原始数据进行TabNet原始模型训练

In [ ]:

from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

tabnet_model = TabNetClassifier()
avg_accuracy =0
tabnet_model.fit(
    X_train=X_train,
    y_train=y_train,
    eval_set=[(X_val,y_val), (X_test, y_test)],
    eval_name=['train', 'valid'],
    eval_metric=['accuracy'],
    max_epochs=50
)
test_preds = tabnet_model.predict(X_test)

    # 计算准确率
acc = accuracy_score(y_test, test_preds)
avg_accuracy += acc
result = confusion_matrix(y_test, test_preds)
print("Confusion Matrix:")
print(result)
result1 = classification_report(y_test, test_preds)
print("Classification Report:", )
print(result1)
print(f"Test accuracy score: {acc}")